In [ ]:
import torch
# import torch.nn as nn
from torch import nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, NeighborSampler, Batch
from torch_geometric.nn import SAGEConv, GAE, TopKPooling
from torch.optim.lr_scheduler import StepLR
from torch.nn import Embedding
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
import gensim.downloader as api
import optuna
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv("./prepro_train_data.csv")

# Make Graph
## Category Embedding

### Age

In [ ]:
age_tensor = torch.tensor(dataset['Age'].values, dtype=torch.float32).unsqueeze(1)

In [ ]:
print("age_tensor.shape >>> ",age_tensor.shape)
print(age_tensor)

### Location

In [ ]:
### Location

le = LabelEncoder()
dataset['Location_encoded'] = le.fit_transform(dataset['Location'])
embedding_layer = Embedding(num_embeddings=151, embedding_dim=29)
location_embeddings = embedding_layer(torch.unsqueeze(torch.tensor(dataset['Location_encoded'].values, dtype=torch.long), dim=1))
location_embeddings = location_embeddings.detach().numpy().squeeze()

In [ ]:
print("location_embeddings.shape >>> ",location_embeddings.shape)
print(location_embeddings)

### Book-Title

In [ ]:
import fasttext

# # 사전 훈련된 FastText 모델 다운로드
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
# !gunzip cc.en.300.bin.gz

In [ ]:
fasttext_model = fasttext.load_model("cc.en.300.bin")

In [ ]:
def get_title_embedding_fasttext(title):
    words = title.split()
    if len(words) == 0:
        return np.zeros(300)
    embeddings = [fasttext_model.get_word_vector(word) for word in words]
    return np.mean(embeddings, axis=0)

In [ ]:
title_embeddings = dataset['Book-Title'].apply(get_title_embedding_fasttext).tolist()

In [ ]:
empty_titles_count = 0
for title in dataset['Book-Title']:
    words = title.split()
    if not words:
        empty_titles_count += 1

print(f"빈 문자열이거나 토큰화된 단어가 없는 책 제목의 개수: {empty_titles_count}")

In [ ]:
for idx, emb in enumerate(title_embeddings):
    if not isinstance(emb, np.ndarray) or emb.shape != (300,):
        print(f"Index: {idx}, Title: {dataset['Book-Title'][idx]}, Embedding: {emb}")

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=20)
title_embeddings_array = np.array(title_embeddings)
reduced_title_embeddings = pca.fit_transform(title_embeddings_array)

In [ ]:
print("reduced_title_embeddings.shape >>> ",reduced_title_embeddings.shape)
print(reduced_title_embeddings)

### Publisher

In [ ]:
### Publisher

le = LabelEncoder()
dataset['Publisher_encoded'] = le.fit_transform(dataset['Publisher'])
embedding_layer = Embedding(num_embeddings=3689, embedding_dim=10)
publisher_embeddings = embedding_layer(torch.unsqueeze(torch.tensor(dataset['Publisher_encoded'].values, dtype=torch.long), dim=1))
publisher_embeddings = publisher_embeddings.detach().numpy().squeeze()

In [ ]:
print("Publisher_embeddings.shape >>> ",Publisher_embeddings.shape)
print(Publisher_embeddings)

### User-ID & Book-ID

In [ ]:
### User-ID & Book-ID

# User-ID 열의 unique한 값들을 리스트로 만들기  > 83256
unique_user_ids = dataset['User-ID'].unique().tolist()
# Book-ID 열의 unique한 값들을 리스트로 만들기 > 243441
unique_book_ids = dataset['Book-ID'].unique().tolist()

# unique_user_ids 리스트를 기반으로 DataFrame 생성
unique_user_ids_df = pd.DataFrame(unique_user_ids, columns=['User-ID'])
# 내림차순 정렬
sorted_unique_user_ids_df = unique_user_ids_df.sort_values(by='User-ID', ascending=True)
# # 인덱스를 새 column으로 추가
sorted_unique_user_ids_df.reset_index(inplace=True, drop=False)
sorted_unique_user_ids_df.rename(columns={'index': 'UserNodeID'}, inplace=True)

# unique_book_ids 리스트를 기반으로 DataFrame 생성
unique_book_ids_df = pd.DataFrame(unique_book_ids, columns=['Book-ID'])
# 내림차순 정렬
sorted_unique_book_ids_df = unique_book_ids_df.sort_values(by='Book-ID', ascending=True)
# #인덱스를 새 column으로 추가
sorted_unique_book_ids_df.reset_index(inplace=True, drop=True)
sorted_unique_book_ids_df.reset_index(inplace=True, drop=False)
sorted_unique_book_ids_df.rename(columns={'index': 'BookNodeID'}, inplace=True)

# 인덱스 시작 번호를 83256으로 설정
sorted_unique_book_ids_df['BookNodeID'] += 83256

# UserNodeID  0 ~ 83255
# BookNodeID   83256 ~ 326696

In [ ]:
def make_ID_dict(df):
    ID_dict = {}
    for index, row in df.iterrows():
        ID_dict[row[1]] = row[0]
    return ID_dict

UserNodeID_dict = make_ID_dict(sorted_unique_user_ids_df)
BookNodeID_dict = make_ID_dict(sorted_unique_book_ids_df)

####### 매핑 진행
dataset['User-ID'] = dataset['User-ID'].map(UserNodeID_dict)
dataset['Book-ID'] = dataset['Book-ID'].map(BookNodeID_dict)

## edge_index

In [ ]:
edge_index = torch.tensor(dataset[['User-ID', 'Book-ID']].values, dtype=torch.long).t().contiguous()

In [ ]:
edge_index

## node feature

In [ ]:
# 사용자 노드와 도서 노드의 총 수 계산:

num_user_nodes = len(unique_user_ids)
num_book_nodes = len(unique_book_ids)

user_ids = dataset["User-ID"].values.tolist()
book_ids = dataset["Book-ID"].values.tolist()

In [ ]:
user_feature_dim = 30  # age (1) + location (29)
book_feature_dim = 30  # title (20) + publisher (10)

user_features = np.zeros((num_user_nodes, user_feature_dim))
book_features = np.zeros((num_book_nodes, book_feature_dim))

for user_id, age, location in zip(user_ids, age_tensor, location_embeddings):
    user_features[user_id] = np.concatenate([age, location], axis=0)

for book_id, title, publisher in zip(book_ids, reduced_title_embeddings, publisher_embeddings):
    book_features[book_id - num_user_nodes] = np.concatenate([title, publisher], axis=0)

node_features = np.vstack((user_features, book_features))

## Weight & Target

In [ ]:
edge_attr = torch.tensor(dataset['Book-Rating'].values, dtype=torch.float).unsqueeze(-1)
y = edge_attr.clone()

# Build Graph

In [ ]:
graph_data = Data(x=torch.tensor(node_features), edge_index=edge_index, edge_attr=edge_attr, y=y)
graph_data.n_id = torch.arange(graph_data.num_nodes)

# Graph Split - Node base

In [ ]:
# 분할
total_edges = edge_index.shape[1]

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

train_idx = train_data.index.values
test_idx = test_data.index.values

# train/test 마스크 초기화
train_mask = torch.zeros(total_edges, dtype=bool)
test_mask = torch.zeros(total_edges, dtype=bool)

# train/test 마스크 설정
train_mask[train_idx] = True
test_mask[test_idx] = True

# 그래프 데이터 생성
data = Data(x=torch.tensor(node_features, dtype=torch.float),
            edge_index=edge_index,
            edge_attr=edge_attr,
            y=y,
            train_mask=train_mask,
            test_mask=test_mask)

data.train_mask = train_mask
data.test_mask = test_mask

# Train/test 마스크가 잘 설정되었는지 확인
print(data.train_mask.sum())  # 결과는 전체 데이터의 약 80%여야 합니다. tensor(697114)
print(data.test_mask.sum())   # 결과는 전체 데이터의 약 20%여야 합니다.tensor(174279)

# UserNodeID  0 ~ 83255
# BookNodeID   83256 ~ 326696

In [ ]:
train_idx_tensor = torch.tensor(train_idx)
test_idx_tensor = torch.tensor(test_idx)

In [ ]:
data.num_nodes = max(data.edge_index.max().item(), len(data.x)) + 1

train_loader = NeighborSampler(data.edge_index, node_idx=train_idx_tensor, sizes=[5, 3], batch_size=2, shuffle=True, num_nodes=data.num_nodes)
test_loader = NeighborSampler(data.edge_index, node_idx=test_idx_tensor, sizes=[5, 3], batch_size=2, shuffle=False, num_nodes=data.num_nodes)

# GraphSAGE

In [ ]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class WeightedSAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(WeightedSAGEConv, self).__init__(aggr='mean')
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index, edge_attr):
        row, col = edge_index
        edge_attr = edge_attr[col]  # 인접 노드에 대한 edge_attr만 선택
        edge_index, edge_attr = add_self_loops(edge_index, edge_attr, num_nodes=x.size(0))
        x = self.lin(x)
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)


    def message(self, x_j, edge_attr):
        return x_j * edge_attr.view(-1, 1)

    def update(self, aggr_out):
        return aggr_out

In [ ]:
import torch
import torch.nn.functional as F

class GraphSAGERegressor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout, activation_name, num_layers=2):
        super(GraphSAGERegressor, self).__init__()

        # Select the activation function
        activations = torch.nn.ModuleDict([
            ['relu', torch.nn.ReLU()],
            ['leaky_relu', torch.nn.LeakyReLU()],
            ['prelu', torch.nn.PReLU()],
            ['elu', torch.nn.ELU()],
            ['silu', torch.nn.SiLU()]
        ])
        self.activation = activations[activation_name]
        if self.activation is None:
            raise ValueError(f"Unsupported activation function: {activation_name}")
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        # Add the first layer
        self.convs.append(WeightedSAGEConv(in_channels, hidden_channels))
        self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        # Add intermediate layers
        for _ in range(num_layers - 2):
            self.convs.append(WeightedSAGEConv(hidden_channels, hidden_channels))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        # Add the last layer
        self.convs.append(WeightedSAGEConv(hidden_channels, out_channels))

        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index_list, edge_attr_list):
        
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index_list[i], edge_attr_list[i])
            x = self.batch_norms[i](x)
            x = self.activation(x)
            x = self.dropout(x)

        x = self.convs[-1](x, edge_index_list[-1], edge_attr_list[-1])

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

in_channels = 30
hidden_channels = 64
out_channels = 1
activation_name = 'relu'
dropout =  0.1

model = GraphSAGERegressor(in_channels, hidden_channels, out_channels, dropout, activation_name, num_layers=2)
model = model.to(device)  # 모델만 옮기고, 데이터는 옮기지 않는다. 그래야 메모리 효율성 업!
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [ ]:
def train(train_loader):
    model.train()
    total_loss = 0
    for batch_size, n_id, adjs in train_loader:
        edge_index_list = [adj.edge_index.to(device) for adj in adjs]
        edge_attr_list = [data.edge_attr.to(device) for _ in adjs]
        
        optimizer.zero_grad()
        out = model(data.x[n_id].to(device), edge_index_list, edge_attr_list)
        predictions = out[:batch_size]
        loss = criterion(predictions, data.y[n_id[data.train_mask][:batch_size]].to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_size, n_id, adjs in test_loader:
            edge_index_list = [adj.edge_index.to(device) for adj in adjs]
            edge_attr_list = [data.edge_attr.to(device) for _ in adjs]
            
            out = model(data.x[n_id].to(device), edge_index_list, edge_attr_list)
            predictions = out[:batch_size]
            loss = criterion(predictions, data.y[n_id[data.test_mask][:batch_size]].to(device))
            total_loss += loss.item()
    return total_loss / len(test_loader)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

in_channels = 30
hidden_channels = 32
out_channels = 1
activation_name = 'relu'
dropout =  0.1

model = GraphSAGERegressor(in_channels, hidden_channels, out_channels, dropout, activation_name, num_layers=2)
model = model.to(device)  # 모델만 옮기고, 데이터는 옮기지 않는다. 그래야 메모리 효율성 업!
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [ ]:
def train(train_loader):
    model.train()
    total_loss = 0
    for batch_size, n_id, adjs in train_loader:
        torch.cuda.empty_cache()
        adjs = [adj.to(device) for adj in adjs]  # Move adjs to device
        edge_index_list = [adj.edge_index for adj in adjs]
        edge_attr_list = [data.edge_attr.to(device) for _ in adjs]  # Reuse edge_attr

        optimizer.zero_grad()
        out = model(data.x[n_id].to(device), edge_index_list, edge_attr_list)
        predictions = out[:batch_size]
        loss = criterion(predictions, data.y[n_id[:batch_size]].to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(test_loader):
    model.eval()
    total_loss = 0
    for batch_size, n_id, adjs in test_loader:
        adjs = [adj.to(device) for adj in adjs]  # Move adjs to device
        edge_index_list = [adj.edge_index for adj in adjs]
        edge_attr_list = [data.edge_attr.to(device) for _ in adjs]  # Reuse edge_attr

        out = model(data.x[n_id].to(device), edge_index_list, edge_attr_list)
        predictions = out[:batch_size]
        loss = criterion(predictions, data.y[n_id[:batch_size]].to(device))
        total_loss += loss.item()
    return total_loss / len(test_loader)

In [ ]:
from tqdm import tqdm

best_val_loss = float('inf')
best_model = None
best_epoch = 0
patience = 2
counter = 0
epoch_num = 10

for epoch in tqdm(range(1, epoch_num+1)):
    train_loss = train(train_loader)
    print(f'Epoch {epoch:02d}, Train Loss: {train_loss:.4f}')
    scheduler.step()
    
    test_loss = test(test_loader)
    print(f'Test Loss: {test_loss:.4f}')

    # Early Stopping
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        best_epoch = epoch
        best_model = model.state_dict()
        torch.save(best_model, 'best_model.pt')  # Save the best model
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping: validation loss did not improve for {patience} epochs.')
            break

print(f'Best validation loss of {best_val_loss:.4f} was achieved at epoch {best_epoch}.')
best_model = torch.load('best_model.pt')  # Load the best model
model.load_state_dict(best_model)
test_loss = test(test_loader)
print(f'Test loss of {test_loss:.4f} was achieved at epoch {best_epoch}.')


In [ ]:
# 하이퍼파라미터 튜닝을 위해 베이지안 최적화 라이브러리인 optuna를 사용

In [ ]:
!pip install optuna

In [ ]:
import optuna

def objective(trial):
    # 하이퍼파라미터 추천값 설정
    hidden_channels = trial.suggest_int('hidden_channels', 32, 64, 128)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    activation_name = trial.suggest_categorical('activation_name', ['relu', 'leaky_relu', 'prelu', 'elu', 'silu'])
    optimizer_name = trial.suggest_categorical('optimizer_name', ['Adam', 'AdamW', 'RMSprop', 'Adagrad'])

    # 모델 및 최적화 생성
    model = GraphSAGERegressor(in_channels, hidden_channels, out_channels, dropout, activation_name).to(device)

    optimizer_class = getattr(torch.optim, optimizer_name)
    optimizer = optimizer_class(model.parameters(), lr=lr, weight_decay=weight_decay)

    # DataLoader 수정
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # 학습 및 평가 루프
    best_test_loss = float('inf')
    for epoch in range(1, 201):
        train_loss = train(train_loader)
        test_loss = test(test_loader)

        if test_loss < best_test_loss:
            best_test_loss = test_loss

    return best_test_loss

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=15)  # n_trials는 원하는 튜닝 횟수에 따라 조정할 수 있습니다.

best_trial = study.best_trial
print(f"Best trial: {best_trial.number}, Test Loss: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")

## Train with Best Parameter

In [ ]:
early_stopping_patience = 10
best_test_loss = float("inf")
epochs_without_improvement = 0

for epoch in range(1, 201):
    train_loss = train(train_loader)
    test_loss = test(test_loader)
    scheduler.step()
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    print(f'Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss}')

    if epochs_without_improvement >= early_stopping_patience:
        print("Early stopping...")
        break

In [ ]:
# Inference
 모델이 처음 보는 유저 또는 도서가 있는 경우, 해당 유저 또는 도서의 노드 특성을 생성하고 기존 그래프 데이터에 추가해야 함!

In [ ]:
def predict_ratings(new_user_book_pairs, user_id_to_index, book_id_to_index):
    model.eval()
    predicted_ratings = []

    with torch.no_grad():
        for user_id, book_id in new_user_book_pairs:
            user_index = user_id_to_index[user_id]
            book_index = book_id_to_index[book_id]

            user_tensor = torch.tensor([user_index], dtype=torch.long, device=device)
            book_tensor = torch.tensor([book_index], dtype=torch.long, device=device)

            edge_index = torch.stack([user_tensor, book_tensor], dim=0)
            edge_attr = torch.tensor([1], dtype=torch.float, device=device)

            out = model(graph_data.x.to(device), edge_index, edge_attr)
            rating = out[user_index, book_index].item()

            predicted_ratings.append((user_id, book_id, rating))

    return predicted_ratings


In [ ]:
predicted_ratings = predict_ratings(new_user_book_pairs, user_id_to_index, book_id_to_index)

print(predicted_ratings)
